<a href="https://colab.research.google.com/github/jacobashwall/Tweets_Classification_Research/blob/main/Suicide_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Model

##Init

In [ ]:
import pandas as pd
import random
import re
import h5py
import math
import numpy as np
from tqdm import tqdm
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.feature_extraction.text import TfidfVectorizer
from tabulate import tabulate
from google.colab import files
import os
import time
from google.colab import drive
import psutil
import fcntl
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import ADASYN
from multiprocessing import Process
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import joblib
import tensorflow as tf
import pickle

In [ ]:
!sudo add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!sudo apt-get update -qq 2>&1 > /dev/null
!sudo apt -y install -qq google-drive-ocamlfuse 2>&1 > /dev/null
!google-drive-ocamlfuse



debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
/usr/bin/xdg-open: 869: www-browser: not found
/usr/bin/xdg-open: 869: links2: not found
/usr/bin/xdg-open: 869: elinks: not found
/usr/bin/xdg-open: 869: links: not found
/usr/bin/xdg-open: 869: lynx: not found
/usr/bin/xdg-open: 869: w3m: not found
xdg-open: no method available for opening 'https://accounts.google.com/o/oauth2/auth?client_id=564921029129.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fgd-ocaml-auth.appspot.com%2Foauth2callback&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&

In [ ]:
!sudo apt-get install -qq w3m # to act as web browser 
!xdg-settings set default-web-browser w3m.desktop # to set default browser
%cd /content
!mkdir drive
%cd drive
!mkdir MyDrive
%cd ..
%cd ..
!google-drive-ocamlfuse /content/drive/MyDrive

debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package w3m.
(Reading database ... 128290 files and directories currently installed.)
Preparing to unpack .../w3m_0.5.3-37ubuntu0.1_amd64.deb ...
Unpacking w3m (0.5.3-37ubuntu0.1) ...
Setting up w3m (0.5.3-37ubuntu0.1) ...
Processing triggers for man-db (2.9.1-1) ...
Processing triggers for mime-support (3.64ubuntu1) ...
/content
/content/drive
/content
/
Access token retrieved correctly.


##Vectors

###Split

In [ ]:
PERCENT = 0.3
DATASETS= ["first" #,"second", "third"
           ]

In [ ]:
def convert_class_to_binary(text):
  if text == "suicide":
    return 1
  else:
    return 0

def write_vectors(dataset):
  print(dataset)
  if dataset == "first":
    print(len(first.index))
    x_train, x_test, y_train, y_test = train_test_split(first["text"], first["is_suicide"], test_size=PERCENT, random_state=42)
  if dataset == "second":
    print(len(second.index))
    x_train, x_test, y_train, y_test = train_test_split(second["text"], second["is_suicide"], test_size=PERCENT, random_state=42)
  if dataset == "third":
    print(len(third.index))
    x_train, x_test, y_train, y_test = train_test_split(third["text"], third["is_suicide"], test_size=PERCENT, random_state=42)

  print(f'train: {len(y_train)}')
  print(f'test: {len(y_test)}')

  x_train = pd.DataFrame(x_train)
  x_test = pd.DataFrame(x_test)
  y_train = pd.DataFrame(y_train)
  y_test = pd.DataFrame(y_test)
  x_train.to_hdf(f'/content/drive/MyDrive/suicide_model_files/data/{dataset}/train_text.h5', key='dataset', mode='w')
  x_test.to_hdf(f'/content/drive/MyDrive/suicide_model_files/data/{dataset}/test_text.h5', key='dataset', mode='w')
  y_train.to_hdf(f'/content/drive/MyDrive/suicide_model_files/data/{dataset}/train_class.h5', key='dataset', mode='w')
  y_test.to_hdf(f'/content/drive/MyDrive/suicide_model_files/data/{dataset}/test_class.h5', key='dataset', mode='w')



In [ ]:
'''
first = pd.read_csv('/content/drive/MyDrive/suicide_model_files/data/1.csv', usecols= ["text","class"] ,encoding='latin-1')
first["class"] = first["class"].apply(convert_class_to_binary)
first= first.rename(columns={"class":"is_suicide"})
first = first.dropna()
'''
first = pd.read_csv('/content/drive/MyDrive/suicide_model_files/data/1_mini.csv', usecols= ["text","label"] ,encoding='latin-1')
first= first.rename(columns={"label":"is_suicide"})
first = first.dropna()

second = pd.read_csv('/content/drive/MyDrive/suicide_model_files/data/2.csv', usecols= ["selftext","is_suicide"],encoding='latin-1')
second= second.rename(columns={"selftext":"text"})
second = second.dropna()

third =  pd.read_csv('/content/drive/MyDrive/suicide_model_files/data/3.csv', usecols= ["tweet","intention"],encoding='latin-1')
third = third.rename(columns={"tweet":"text"})
third = third.rename(columns={"intention":"is_suicide"})
third = third.dropna()

for ds in DATASETS:
  write_vectors(ds)


first
20000
train: 14000
test: 6000


###TF-IDF

In [ ]:
TOKEN_PATTERN="[^ \n]+" # baseline tokenize, only by space and end of row
DATASETS= ["first",
           #"second",
           #"third"
           ]
'''
SPECIAL_CHARCATERS_REMOVAL=r"(?u)\b\w+[\"\'״]*\w\b" # tokenize by " " and "/n" and allow abbreviations

# Gets all the stop words from the file
pattern = re.compile(SPECIAL_CHARCATERS_REMOVAL)
with open("/content/drive/MyDrive/model_files/data/stop_words.txt", 'r') as f:
        STOP_WORDS = pattern.findall(f.read())

# Insert all the abbreviations into a dictionary
ABBREVIATIONS = {}
with open("/content/drive/MyDrive/model_files/data/abbreviations.txt") as f:
    for line in f:
        key, value = line.split("-")
        ABBREVIATIONS[key.strip()] = value.strip()

# To preprocess open abbrevaitons
def expand_abbreviations(text):
    """
    a function that gets a text and replace all abbreviations with thier expnaded form
    """
    for match in re.finditer(SPECIAL_CHARCATERS_REMOVAL, text):
        abbreviation = match.group()
        normelized_abbreviation=abbreviation
        # normalize all " 
        if "״" in abbreviation:
          normelized_abbreviation=abbreviation.replace("״","\"")
        if "''" in abbreviation:
          normelized_abbreviation=abbreviation.replace("''","\"")
        # if its in the, replace with expanded form
        if normelized_abbreviation in ABBREVIATIONS:
            text = text.replace(abbreviation, ABBREVIATIONS[normelized_abbreviation])
    return text
'''

'\nSPECIAL_CHARCATERS_REMOVAL=r"(?u)\x08\\w+["\'״]*\\w\x08" # tokenize by " " and "/n" and allow abbreviations\n\n# Gets all the stop words from the file\npattern = re.compile(SPECIAL_CHARCATERS_REMOVAL)\nwith open("/content/drive/MyDrive/model_files/data/stop_words.txt", \'r\') as f:\n        STOP_WORDS = pattern.findall(f.read())\n\n# Insert all the abbreviations into a dictionary\nABBREVIATIONS = {}\nwith open("/content/drive/MyDrive/model_files/data/abbreviations.txt") as f:\n    for line in f:\n        key, value = line.split("-")\n        ABBREVIATIONS[key.strip()] = value.strip()\n\n# To preprocess open abbrevaitons\ndef expand_abbreviations(text):\n    """\n    a function that gets a text and replace all abbreviations with thier expnaded form\n    """\n    for match in re.finditer(SPECIAL_CHARCATERS_REMOVAL, text):\n        abbreviation = match.group()\n        normelized_abbreviation=abbreviation\n        # normalize all " \n        if "״" in abbreviation:\n          normelize

In [ ]:
def read_train_text(dataset):
  store = pd.HDFStore(f'/content/drive/MyDrive/suicide_model_files/data/{dataset}/train_text.h5')
  df = store.select('dataset')
  store.close()

  columns_to_include = ["text"]
  df = df[columns_to_include]
  df.dropna(subset=["text"], inplace=True)
  return df
def get_words(texts,preprocessing):
  """
  returns all the tweets words by the preprocessing method
  """
  corpus=texts
  '''
  if "O" in preprocessing:
    corpus=corpus.apply(expand_abbreviations)
  
  if "C" in preprocessing:
    pattern = re.compile(SPECIAL_CHARCATERS_REMOVAL)
  else:
  '''
  pattern = re.compile(TOKEN_PATTERN)

  corpus_words=[]
  for document in tqdm(corpus):
      corpus_words+= pattern.findall(document)
  return corpus_words

def get_terms():
  columns= ["nt", "f", "tf","idf","tfidf"]
  meanings= ["Number of different documents that the word appears in.",
   "Number of appearances of the word in all documents.",
   "Term frequency.",
   "Inverse document frequency.",
   "Term fruquency multiplied by inverse document frequency."]
  return columns, meanings
   
  
def write_tf_idf_chart(preprocessing, vectorizer, dataset):
  """
  writes the tf idf chart by the given preprocessing vectorizer
  """
  train_texts = read_train_text(dataset)
  sparse_matrix = vectorizer.fit_transform(train_texts["text"])

  # Get the vocabulary from the vectorizer object
  vocab = vectorizer.get_feature_names_out()

  df = pd.DataFrame(columns=['word', 'nt', 'f', 'tf', 'idf', 'tf-idf',' ','Column','Meaning'])
  # Gather all the texts words to calculate words frequency
  print("Getting words from texts:")
  texts_words = get_words(train_texts["text"], preprocessing)
  print("done!")
  print(len(vocab))

  print("calculating 'nt' and 'f':")
  for i, word in tqdm(enumerate(vocab)):
      # The column of tf-idf values of the specific word, as the rows are the texts 
      word_column = sparse_matrix.getcol(i)
      # Count the number of texts the word appears in (get no zero = getnz)
      nt = word_column.getnnz()
      # Count the number of times the word is used in all texts
      f = texts_words.count(word)
      # append all the data we collected so far to each word to the data frame
      new_df = pd.DataFrame({'word': word, 'nt': nt, 'f': f, 'tf': 0, 'idf': 0, 'tf-idf': 0}, index=[0])

      df = pd.concat([df, new_df], ignore_index=True)
  print("done!")

  # the number of words in total is the sum of the f of each word
  sig_f = df['f'].sum()

  print("calculating 'tf','idf' and 'tf-idf':")
  for i, word in tqdm(enumerate(vocab)):
      # the index of the word
      index = df.index[df['word'] == word][0]
      f = df.at[index, 'f']
      nt = df.at[index, 'nt']
      # Calculate the tf value
      tf = f / sig_f
      # shape[0] is the number of rows, aka the number of texts
      n = sparse_matrix.shape[0]
      # Calculate the idf value
      idf = np.log(n / (nt + 1))
      # Calculate the tf-idf value
      tf_idf = tf * idf
      # update the word's remaining information to the dataframe
      df.at[index, 'tf'] = tf
      df.at[index, 'idf'] = idf
      df.at[index, 'tf-idf'] = tf_idf
  print("done!")

  # sort in descending order
  df = df.sort_values('tf-idf', ascending=False)
  # add vocabulary
  columns, meanings = get_terms()
  for i in range(len(columns)):
    df.at[df.index[i], "Column"] = columns[i]
    df.at[df.index[i], "Meaning"] = meanings[i]
  # write to csv
  df.to_csv(f'/content/drive/MyDrive/suicide_model_files/tf idf charts/{dataset}/{preprocessing}_words_tfidf_chart.csv', encoding='latin-1',index=False)
  # write to h5
  df.to_hdf(f'/content/drive/MyDrive/suicide_model_files/tf idf charts/{dataset}/{preprocessing}_words_tfidf_chart.h5', key='words_df', mode='w')

In [ ]:
for ds in DATASETS:
  nt = 0
  if ds == 'first':
    nt = 10
  if ds == 'second':
    nt = 2
  if ds == 'third':
    nt = 3
  VECTORIZERS={ # the tf-idf vectorizers by preprocessing method
    #'N' : TfidfVectorizer(min_df=nt,lowercase=False, token_pattern=TOKEN_PATTERN),#None, baseline
    #'3gram' : TfidfVectorizer(min_df=nt,lowercase=False, analyzer='char', ngram_range=(3, 3)),
    #'4gram' : TfidfVectorizer(min_df=nt,lowercase=False, analyzer='char', ngram_range=(4, 4)),
    '5gram' : TfidfVectorizer(min_df=nt,lowercase=False, analyzer='char', ngram_range=(5, 5)),

    # only one preprocessing method
    #'S':TfidfVectorizer(min_df=NT,lowercase=False, token_pattern=TOKEN_PATTERN,stop_words=STOP_WORDS),#Stop word removal
    #'C':TfidfVectorizer(min_df=NT,lowercase=False, token_pattern=SPECIAL_CHARCATERS_REMOVAL),#special Characters removal
    #'O':TfidfVectorizer(min_df=NT,lowercase=False, token_pattern=TOKEN_PATTERN, preprocessor=expand_abbreviations),#Open abbreviations
    # pairing preprocssing methods
    #'SC':TfidfVectorizer(min_df=NT,lowercase=False, token_pattern=SPECIAL_CHARCATERS_REMOVAL,stop_words=STOP_WORDS),
    #'SO':TfidfVectorizer(min_df=NT,lowercase=False, token_pattern=TOKEN_PATTERN,stop_words=STOP_WORDS,preprocessor=expand_abbreviations),
    #'CO':TfidfVectorizer(min_df=NT,lowercase=False, token_pattern=SPECIAL_CHARCATERS_REMOVAL,preprocessor=expand_abbreviations),
    # all preprocessing methods
    #'SCO':TfidfVectorizer(min_df=NT,lowercase=False, token_pattern=SPECIAL_CHARCATERS_REMOVAL,stop_words=STOP_WORDS,preprocessor=expand_abbreviations),
}
  print(f'writing charts for the {ds} dataset.')
  #iterate over each preprocessing method and writes the tfidf chart
  for preprocessing, vectorizer in VECTORIZERS.items():
    print("------------------------------------------")
    print(f"writing the {preprocessing} tf-idf chart:")
    write_tf_idf_chart(preprocessing, vectorizer,ds)
    print("------------------------------------------")

writing charts for the first dataset.
------------------------------------------
writing the 5gram tf-idf chart:
Getting words from texts:


100%|██████████| 14000/14000 [00:00<00:00, 32434.87it/s]


done!
60621
calculating 'nt' and 'f':


60621it [2:51:38,  5.89it/s]


done!
calculating 'tf','idf' and 'tf-idf':


60621it [08:16, 122.12it/s]


done!


<ipython-input-5-7d5ee468499c>:104: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block0_values] [items->Index(['word', 'nt', 'f', 'tf', 'idf', 'tf-idf', ' ', 'Column', 'Meaning'], dtype='object')]

  df.to_hdf(f'/content/drive/MyDrive/suicide_model_files/tf idf charts/{dataset}/{preprocessing}_words_tfidf_chart.h5', key='words_df', mode='w')


------------------------------------------


###Vectorize

In [ ]:
DATASETS= ["first" ,
           #"second", "third"
           ]
NUMBER_OF_WORDS = [5000,4000,3000,2000,1000]  # the number of the highest tfidf words
TOKEN_PATTERN="[^ \n]+" # baseline tokenize, only by space and end of row

'''
SPECIAL_CHARCATERS_REMOVAL=r"(?u)\b\w+[\"\'״]*\w\b" # tokenize by " " and "/n" and allow abbreviations

# Gets all the stop words from the file
with open("/content/drive/MyDrive/model_files/data/stop_words.txt", 'r') as f:
        STOP_WORDS = f.read().splitlines()

# Insert all the abbreviations into a dictionary
ABBREVIATIONS = {}
with open("/content/drive/MyDrive/model_files/data/abbreviations.txt") as f:
    for line in f:
        key, value = line.split("-")
        ABBREVIATIONS[key.strip()] = value.strip()

# To preprocess open abbrevaitons
def expand_abbreviations(text):
    """
    a function that gets a text and replace all abbreviations with thier expnaded form
    """
    for match in re.finditer(SPECIAL_CHARCATERS_REMOVAL, text):
        abbreviation = match.group()
        normelized_abbreviation=abbreviation
        # normalize all " 
        if "״" in abbreviation:
          normelized_abbreviation=abbreviation.replace("״","\"")
        if "''" in abbreviation:
          normelized_abbreviation=abbreviation.replace("''","\"")
        # if its in the, replace with expanded form
        if normelized_abbreviation in ABBREVIATIONS:
            text = text.replace(abbreviation, ABBREVIATIONS[normelized_abbreviation])
    return text
'''

'\nSPECIAL_CHARCATERS_REMOVAL=r"(?u)\x08\\w+["\'״]*\\w\x08" # tokenize by " " and "/n" and allow abbreviations\n\n# Gets all the stop words from the file\nwith open("/content/drive/MyDrive/model_files/data/stop_words.txt", \'r\') as f:\n        STOP_WORDS = f.read().splitlines()\n\n# Insert all the abbreviations into a dictionary\nABBREVIATIONS = {}\nwith open("/content/drive/MyDrive/model_files/data/abbreviations.txt") as f:\n    for line in f:\n        key, value = line.split("-")\n        ABBREVIATIONS[key.strip()] = value.strip()\n\n# To preprocess open abbrevaitons\ndef expand_abbreviations(text):\n    """\n    a function that gets a text and replace all abbreviations with thier expnaded form\n    """\n    for match in re.finditer(SPECIAL_CHARCATERS_REMOVAL, text):\n        abbreviation = match.group()\n        normelized_abbreviation=abbreviation\n        # normalize all " \n        if "״" in abbreviation:\n          normelized_abbreviation=abbreviation.replace("״",""")\n        

In [ ]:
def read_train_text(dataset):
  store = pd.HDFStore(f'/content/drive/MyDrive/suicide_model_files/data/{dataset}/train_text.h5')
  df = store.select('dataset')
  store.close()

  columns_to_include = ["text"]
  df = df[columns_to_include]
  df.dropna(subset=["text"], inplace=True)
  return df

def read_test_text(dataset):
  store = pd.HDFStore(f'/content/drive/MyDrive/suicide_model_files/data/{dataset}/test_text.h5')
  df = store.select('dataset')
  store.close()

  columns_to_include = ["text"]
  df = df[columns_to_include]
  df.dropna(subset=["text"], inplace=True)
  return df


def read_train_class(dataset):
  """
  reads the data from the train knesst members sheet
  :return: the data as pandas data set
  """
  store = pd.HDFStore(f'/content/drive/MyDrive/suicide_model_files/data/{dataset}/train_class.h5')
  df = store.select('dataset')
  store.close()

  columns_to_include = "is_suicide"
  df = df[columns_to_include]
  df.dropna( inplace=True)
  return df

def read_test_class(dataset):
  """
  reads the data from the test knesst members sheet
  :return: the data as pandas data set
  """
  store = pd.HDFStore(f'/content/drive/MyDrive/suicide_model_files/data/{dataset}/test_class.h5')
  df = store.select('dataset')
  store.close()

  columns_to_include = "is_suicide"
  df = df[columns_to_include]
  df.dropna( inplace=True)
  return df

def write_classification(dataset):
  """
  return the tweet vector with the classification
  :param tweets: the tweets set
  :param knesset_members_train_set: the knesset members information
  :param classification: the desired classification
  :return: the classified tweets dataframe
  """
  train_class=read_train_class(dataset)
  test_class=read_test_class(dataset)

  #df.to_csv(f'/content/drive/MyDrive/model_files/classification/{set}_tweets_classification_{classification}.csv', encoding='utf-8-sig', index=False)
  train_class.to_hdf(f'/content/drive/MyDrive/suicide_model_files/classification/{dataset}/train_text_classification.h5', key='classification', mode='w')
  test_class.to_hdf(f'/content/drive/MyDrive/suicide_model_files/classification/{dataset}/test_text_classification.h5', key='classification', mode='w')


def write_vectors(tfidf_vocabulary,vectors,set,amount,preprocessing,dataset):
  """
  write the given set tfidf vectors by the tf idf vocabulary
  """
  dense_vectors = vectors.toarray()
  df = pd.DataFrame(dense_vectors, columns=tfidf_vocabulary)
  #df.to_csv(f'/content/drive/MyDrive/suicide_model_files/vectors/{dataset}/{preprocessing}_{set}_tweets_vectors_{amount}.csv', encoding='utf-8-sig',index=False)
  df.to_hdf(f'/content/drive/MyDrive/suicide_model_files/vectors/{dataset}/{preprocessing}_{set}_text_vectors_{amount}.h5', key='vectors', mode='w')

def get_words(amount,preprocessing,dataset):
  """
  get the highest tf-idf words by a given amount from the df-idf chart
  :param amount: the amount of words
  :return: the given amount of words with the highest TF-IDF value
  """
  words = pd.read_hdf(f'/content/drive/MyDrive/suicide_model_files/tf idf charts/{dataset}/{preprocessing}_words_tfidf_chart.h5',key="words_df")
  words = words.head(amount)
  words = words['word'].tolist()
  return words

In [ ]:
for ds in DATASETS:
  print(ds)
  train_texts = read_train_text(ds)
  test_texts = read_test_text(ds)
  for amount in NUMBER_OF_WORDS:
    VECTORIZERS={ # the tf-idf vectorizers by preprocessing method
      'N' : TfidfVectorizer(vocabulary=get_words(amount,'N',ds),lowercase=False, token_pattern=TOKEN_PATTERN),#None, baseline
      '3gram' : TfidfVectorizer(vocabulary=get_words(amount,'3gram',ds),lowercase=False, analyzer='char', ngram_range=(3, 3)),
      '4gram' : TfidfVectorizer(vocabulary=get_words(amount,'4gram',ds),lowercase=False, analyzer='char', ngram_range=(4, 4)),
      '5gram' : TfidfVectorizer(vocabulary=get_words(amount,'5gram',ds),lowercase=False, analyzer='char', ngram_range=(5, 5)),

    # only one preprocessing method
      #'S':TfidfVectorizer(vocabulary=get_words(amount,'S'),lowercase=False, token_pattern=TOKEN_PATTERN,stop_words=STOP_WORDS),#Stop word removal
      #'C':TfidfVectorizer(vocabulary=get_words(amount,'C'),lowercase=False, token_pattern=SPECIAL_CHARCATERS_REMOVAL),#special Characters removal
      #'O':TfidfVectorizer(vocabulary=get_words(amount,'O'),lowercase=False, token_pattern=TOKEN_PATTERN, preprocessor=expand_abbreviations),#Open abbreviations
    # pairing preprocssing methods
    #'SC':TfidfVectorizer(vocabulary=get_words(amount,'SC'),lowercase=False, token_pattern=SPECIAL_CHARCATERS_REMOVAL,stop_words=STOP_WORDS),
    #'SO':TfidfVectorizer(vocabulary=get_words(amount,'SO'),lowercase=False, token_pattern=TOKEN_PATTERN,stop_words=STOP_WORDS,preprocessor=expand_abbreviations),
    #'CO':TfidfVectorizer(vocabulary=get_words(amount,'CO'),lowercase=False, token_pattern=SPECIAL_CHARCATERS_REMOVAL,preprocessor=expand_abbreviations),
    # all preprocessing methods
    #'SCO':TfidfVectorizer(vocabulary=get_words(amount,'SCO'),lowercase=False, token_pattern=SPECIAL_CHARCATERS_REMOVAL,stop_words=STOP_WORDS,preprocessor=expand_abbreviations),
  }
    for preprocessing, vectorizer in tqdm(VECTORIZERS.items()):
      print(preprocessing)
      tfidf_vocabulary=get_words(amount,preprocessing,ds)
      train_vectors = vectorizer.fit_transform(train_texts["text"])
      write_vectors(tfidf_vocabulary,train_vectors,"train",amount,preprocessing,ds)

      test_vectors = vectorizer.fit_transform(test_texts["text"])
      write_vectors(tfidf_vocabulary,test_vectors,"test",amount,preprocessing,ds)
  

first


  0%|          | 0/4 [00:00<?, ?it/s]

N


 25%|██▌       | 1/4 [01:08<03:25, 68.53s/it]

3gram


...

 50%|█████     | 2/4 [00:34<00:34, 17.04s/it]

4gram


 75%|███████▌  | 3/4 [00:49<00:16, 16.14s/it]

5gram


100%|██████████| 4/4 [01:04<00:00, 16.21s/it]


In [ ]:
for ds in DATASETS:
    write_classification(ds)   



##ML Methods

In [ ]:
RANDOM_STATE = 1 # in order to get consistent results for the random forest algorithem
DATASETS= ["first" ,"second", "third"]

NUM_OF_WORDS=[1000,2000,3000,4000,5000]
ML_CLASSIFIERS= [ # the classifers we use
"RF"
,"SVM"
,"MNB"
,"LR"
,"MLP"
]
OVERSAMPLERS=["none",
              #"random",
              #"smote",
              #"adasyn"
              ]
# the preprocessing methods
PREPROCESSORS=[#'N',
               '3gram',
               #'4gram',
               #'5gram',
]
               #,'S','C','O','SC','SO','CO','SCO']
RANDOM_STATE=42

In [ ]:
def get_sets_vectors(amount,preprocessing,oversampler,dataset):
  """
  reads and returns the vectors and classifcation of the train and test sets.
  """
  x_train=pd.read_hdf(f'/content/drive/MyDrive/suicide_model_files/vectors/{dataset}/{preprocessing}_train_text_vectors_{amount}.h5',key='vectors')
  x_train.columns = x_train.columns.astype(str)
  y_train=pd.read_hdf(f'/content/drive/MyDrive/suicide_model_files/classification/{dataset}/train_text_classification.h5',key='classification')
  y_train=np.array(y_train).astype(int)  
  x_test=pd.read_hdf(f'/content/drive/MyDrive/suicide_model_files/vectors/{dataset}/{preprocessing}_test_text_vectors_{amount}.h5',key='vectors')
  x_test.columns = x_test.columns.astype(str)
  y_test=pd.read_hdf(f'/content/drive/MyDrive/suicide_model_files/classification/{dataset}/test_text_classification.h5',key='classification')
  y_test=np.array(y_test).astype(int)


  if oversampler != "none":
    if oversampler == "random":
      oversampler = RandomOverSampler(random_state=RANDOM_STATE)
    if oversampler == "smote":
      oversampler = SMOTE(random_state=RANDOM_STATE)
    if oversampler == "adasyn":
      oversampler = ADASYN(random_state=RANDOM_STATE)

    x_train, y_train = oversampler.fit_resample(x_train, y_train)      
  return x_train, x_test, y_train, y_test


def score_classifier(y_test, predictions, classifier,classifiers_scores):
  """
  returns the given classifers score by comparing the predictions and true classifies.
  """
  accuracy = accuracy_score(y_test, predictions)
  classifiers_scores[classifier] = accuracy


def run_classifier(x_train, x_test, y_train, y_test,classifier,classifiers_scores):
  """
  runs the specified classifier with the given train and test sets.
  """
  # create a scikit classifier by the given classifier name
  if classifier=="RF":
      clf = RandomForestClassifier(random_state=RANDOM_STATE)
  elif classifier=="SVM":
      clf = SVC()
  elif classifier=="MLP":
      clf = MLPClassifier()
  elif classifier=="MNB":
      clf = MultinomialNB()
  elif classifier=="LR":
      clf = LogisticRegression()
  else:
      raise ValueError("unknown classifier given")
  clf.fit(x_train, y_train)
  predictions = clf.predict(x_test)
  score_classifier(y_test, predictions, classifier,classifiers_scores)
  return clf


def ml_classifiers(x_train, x_test, y_train, y_test,classifiers):
  """
  creates the classifers and prints their scores. 
  """
  # scores dictionaries to store and summary by measure
  classifiers_scores = {}
  for classifier in tqdm(classifiers):
    run_classifier(x_train, x_test, y_train, y_test,classifier,classifiers_scores)

  return classifiers_scores

def create_score_chart(oversampling,preprocessing,dataset):
  df_columns = []
  # Create a data frame with the title row
  df_columns.append("Number of words")
  df_columns = df_columns + ML_CLASSIFIERS
  df = pd.DataFrame(columns=df_columns)
    
  # Create an array of empty strings
  empty_strings = np.empty(len(df_columns), dtype='U64')
  empty_strings[:] = ''

  # Set the first element of the array to a specific value
  empty_strings[0] = oversampling+ " oversampling after preprocess by " + preprocessing
  title = pd.DataFrame(data=[empty_strings], columns=df.columns)
  for amount in NUM_OF_WORDS:
      print(f"running ml methods for {amount} of words with preprocessing '{preprocessing}'...")
      x_train, x_test, y_train, y_test=get_sets_vectors(amount,preprocessing,oversampling,dataset)
      classifiers_scores = ml_classifiers(x_train, x_test, y_train, y_test,ML_CLASSIFIERS)
      print(classifiers_scores)
      df_row = []
      df_row.append(amount)
      df_row = df_row + list(classifiers_scores.values())
      df.loc[len(df.index)] = df_row
  print(tabulate(df, headers='keys', tablefmt='psql'))
  # Concatenate the two data frames
  print(oversampling+" "+preprocessing+":")
  header = pd.DataFrame(data=[df_columns], columns=df_columns)
  csv_df = pd.concat([title,header,df], ignore_index=True)
  csv_df.to_csv(f'/content/drive/MyDrive/suicide_model_files/results/{dataset}/{oversampling}_oversampling_ of_{preprocessing}_results.csv', header=False, encoding='utf-8-sig', index=False)
  df.to_hdf(f'/content/drive/MyDrive/suicide_model_files/results/{dataset}/{oversampling}_oversampling_ of_{preprocessing}_results.h5', key='results', mode='w')
  print("finished writing "+oversampling+" oversampling by "+preprocessing+":")

In [ ]:
for ds in DATASETS:
  for oversampling in OVERSAMPLERS:
    processes = [] #open a process for each preprocessing method
    for preprocessing in PREPROCESSORS:
      p = Process(target=create_score_chart, args=(oversampling,preprocessing,ds))
      p.start()
      processes.append(p)

    # wait for all processes to finish
    for p in processes:
      p.join()

running ml methods for 1000 of words with preprocessing '3gram'...


  0%|          | 0/5 [00:00<?, ?it/s]

RF
0.6133567662565905


 20%|██        | 1/5 [00:00<00:02,  1.50it/s]

SVM
0.6379613356766256


...

LR
0.9133771929824561


 80%|████████  | 4/5 [01:17<00:14, 14.92s/it]

MLP
0.9057017543859649


100%|██████████| 5/5 [02:33<00:00, 30.75s/it]


{'RF': 0.9093567251461988, 'SVM': 0.9301900584795322, 'MNB': 0.8676900584795322, 'LR': 0.9133771929824561, 'MLP': 0.9057017543859649}
+----+-------------------+----------+----------+----------+----------+----------+
|    |   Number of words |       RF |      SVM |      MNB |       LR |      MLP |
|----+-------------------+----------+----------+----------+----------+----------|
|  0 |              1000 | 0.887427 | 0.910819 | 0.826754 | 0.888158 | 0.885599 |
|  1 |              2000 | 0.89364  | 0.920322 | 0.848684 | 0.896199 | 0.888158 |
|  2 |              3000 | 0.903874 | 0.91886  | 0.855994 | 0.906433 | 0.89693  |
|  3 |              4000 | 0.902412 | 0.925073 | 0.86769  | 0.910819 | 0.89693  |
|  4 |              5000 | 0.909357 | 0.93019  | 0.86769  | 0.913377 | 0.905702 |
+----+-------------------+----------+----------+----------+----------+----------+
none 3gram:
finished writing none oversampling by 3gram:


##Summary

In [ ]:
DATASETS= ["first" ,"second", "third"]
PREPROCESSORS=['N',
               '3gram',
               '4gram',
               '5gram'
               #'S','C','O'
               ]#,'SC','SO','CO','SCO']
ML_CLASSIFIERS= [ # the classifers we use
"RF"
,"SVM"
,"MNB"
,"LR"
,"MLP"
]
NUM_OF_WORDS=[1000,2000,3000,4000,5000]

OVERSAMPLERS=["none",
              #"random",
              #"smote",
              #"adasyn"
              ]

In [ ]:
df_columns = ["Feature type","Classifier","Number of features","Score"]
for oversampling in OVERSAMPLERS:
  for ds in DATASETS:
    print(f"starting {ds} dataset:")
    summary = pd.DataFrame(columns=df_columns)
    for preprocessing in PREPROCESSORS:
        ml_results = pd.read_hdf(f'/content/drive/MyDrive/suicide_model_files/results/{ds}/{oversampling}_oversampling_ of_{preprocessing}_results.h5', key='results')
        max_value_index = ml_results[ml_results.columns[1:]].stack().idxmax()
        row_index= int(max_value_index[0])
        column_index= ml_results.columns.get_loc(max_value_index[1])
        row={"Feature type": preprocessing,"Classifier": max_value_index[1],"Number of features": ml_results.iat[row_index,0],"Score":ml_results.iat[row_index,column_index] }
        summary.loc[len(summary.index)] = row
    summary = summary.sort_values('Score', ascending=False)
    print(tabulate(summary, headers='keys', tablefmt='psql',showindex=False))
    summary.to_csv(f'/content/drive/MyDrive/suicide_model_files/summary/datasets/{ds}/results_summary.csv', index=False)
    summary.to_hdf(f'/content/drive/MyDrive/suicide_model_files/summary/datasets/{ds}/results_summary.h5', key='summary', mode='w') 


starting first dataset:
+----------------+--------------+----------------------+----------+
| Feature type   | Classifier   |   Number of features |    Score |
|----------------+--------------+----------------------+----------|
| 3gram          | SVM          |                 5000 | 0.9245   |
| N              | SVM          |                 4000 | 0.918333 |
| 4gram          | SVM          |                 5000 | 0.910833 |
| 5gram          | SVM          |                 5000 | 0.894667 |
+----------------+--------------+----------------------+----------+
starting second dataset:
+----------------+--------------+----------------------+----------+
| Feature type   | Classifier   |   Number of features |    Score |
|----------------+--------------+----------------------+----------|
| 4gram          | SVM          |                 1000 | 0.688928 |
| N              | MNB          |                 5000 | 0.683656 |
| 3gram          | MNB          |                 4000 | 0.681898 |

In [ ]:
df_columns = ["Dataset","Feature type","Classifier","Number of features","Score"]
oversampling = 'none'
model_summary = pd.DataFrame(columns=df_columns)
for ds in DATASETS:
      summary= pd.read_hdf(f'/content/drive/MyDrive/suicide_model_files/summary/datasets/{ds}/results_summary.h5', key='summary')
      summary["Dataset"] = ds
      model_summary = pd.concat([model_summary,summary], ignore_index=True)

model_summary = model_summary.sort_values('Score', ascending=False)
model_summary = model_summary.head(4)
print(tabulate(model_summary, headers='keys', tablefmt='psql',showindex=False))
model_summary.to_hdf(f'/content/drive/MyDrive/suicide_model_files/summary/baseline_results_summary.h5',key='model_summary', mode='w')
model_summary.to_csv(f'/content/drive/MyDrive/suicide_model_files/summary/baseline_results_summary.csv', encoding='utf-8-sig', index=False)


 

+-----------+----------------+--------------+----------------------+----------+
| Dataset   | Feature type   | Classifier   |   Number of features |    Score |
|-----------+----------------+--------------+----------------------+----------|
| third     | 3gram          | SVM          |                 5000 | 0.93019  |
| third     | N              | SVM          |                 4000 | 0.927997 |
| third     | 4gram          | SVM          |                 4000 | 0.919956 |
| third     | 5gram          | SVM          |                 5000 | 0.899123 |
+-----------+----------------+--------------+----------------------+----------+


##Tuning

In [ ]:
def get_words(amount,preprocessing,dataset):
  """
  get the highest tf-idf words by a given amount from the df-idf chart
  :param amount: the amount of words
  :return: the given amount of words with the highest TF-IDF value
  """
  words = pd.read_hdf(f'/content/drive/MyDrive/suicide_model_files/tf idf charts/{dataset}/{preprocessing}_words_tfidf_chart.h5',key="words_df")
  words = words.head(amount)
  words = words['word'].tolist()
  return words

model_summary = pd.read_hdf(f'/content/drive/MyDrive/suicide_model_files/summary/baseline_results_summary.h5', key='model_summary')

amount = int(model_summary.iloc[0]['Number of features'])
preprocessing = model_summary.iloc[0]['Feature type']
oversampling = 'none'
ds = model_summary.iloc[0]['Dataset']
ml_method = model_summary.iloc[0]['Classifier']
score = model_summary.iloc[0]['Score']
print('start training:')
print('amount: ' + str(amount))
print('features type: '+ str(preprocessing))
print('oversampling: '+ str(oversampling))
print('dataset: '+ str(ds))
print('ml method: '+ str(ml_method))
print('score before tuning: ' +str(score) )

x_train, x_test, y_train, y_test=get_sets_vectors(amount,preprocessing,oversampling,ds)

TOKEN_PATTERN="[^ \n]+" # baseline tokenize, only by space and end of row
VECTORIZERS={ # the tf-idf vectorizers by preprocessing method
      'N' : TfidfVectorizer(vocabulary=get_words(amount,'N',ds),lowercase=False, token_pattern=TOKEN_PATTERN),#None, baseline
      '3gram' : TfidfVectorizer(vocabulary=get_words(amount,'3gram',ds),lowercase=False, analyzer='char', ngram_range=(3, 3)),
      '4gram' : TfidfVectorizer(vocabulary=get_words(amount,'4gram',ds),lowercase=False, analyzer='char', ngram_range=(4, 4)),
      '5gram' : TfidfVectorizer(vocabulary=get_words(amount,'5gram',ds),lowercase=False, analyzer='char', ngram_range=(5, 5)),
}
vectorizer = VECTORIZERS[preprocessing]
vectorizer.fit_transform(x_train)

param_grid={}
if ml_method == "SVM":
  clf = SVC()
  #clf.fit(x_train,y_train)
  param_grid = {'C': [1], 'kernel': ['linear', 'poly','rbf'], 'degree': [2, 3, 4],'gamma': [10, 100, 1000]}

x = pd.concat([x_train,x_test], axis = 0)
y = np.concatenate([ y_train, y_test])
grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=5)
grid_search.fit(x, y)

best_params = grid_search.best_params_
best_score = grid_search.best_score_
print('tuned params:')
print(best_params)
print('best score:')
print(best_score)



pickle.dump(grid_search, open('/content/drive/MyDrive/suicide_model_files/model.sav', 'wb'))
pickle.dump(vectorizer, open('/content/drive/MyDrive/suicide_model_files/vectorizer.sav', 'wb'))

print('done!')

start training:
amount: 5000
features type: 3gram
oversampling: none
dataset: third
ml method: SVM
score before tuning: 0.9301900584795322


##Usage

In [ ]:
print('booting server.')
model = pickle.load(open('/content/drive/MyDrive/suicide_model_files/model.sav', 'rb'))
vectorizer = pickle.load(open('/content/drive/MyDrive/suicide_model_files/vectorizer.sav', 'rb'))
user_input = ""
print('server is up.')
while True:
  user_input = input("Please enter an input, type EXIT to leave:")
  if user_input == 'EXIT':
    break
  input_vector = vectorizer.transform([user_input]).toarray()
  prediction = model.predict(input_vector)
  if(prediction[0]):
    print("NO!!!!!! you are still young")
  else:
    print("yeeeeee, idc")

booting server.
server is up.


/usr/local/lib/python3.9/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(


NO!!!!!! you are still young


/usr/local/lib/python3.9/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(


yeeeeee, idc


/usr/local/lib/python3.9/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(


NO!!!!!! you are still young
